In [1]:
import jdc
import re
import copy
# import importlib
# importlib.reload(stack_and_queue)
from stack_and_queue import Stack, Queue

In [2]:
def divide(arr, idx, l, r):
    """
    Partition the list
    helper function for quick sort
    
    Args:
        arr: the list of tuples to be divided
        idx: the index of tuple element to be sorted by
        l: left index of the sublist to be divided
        r: right index of the sublist to be divided

    Return:
        The final index of the pivot
    """
    pivot = arr[r][idx] # choose the right most element as pivot
    i, j = l, r-1
    while i < j:
        while arr[i][idx] >= pivot and i <= j:
            # iterate from left to right until an element >= pivot is found
            i += 1
        while arr[j][idx] < pivot and i < j:
            # iterate from right to left until an element < pivot is found
            j -= 1
        if i<j:
            # swap elements so the element < pivot moves to the left of pivot and the larger one right
            arr[i], arr[j] = arr[j], arr[i]
    if arr[i][idx] < arr[r][idx]:
        # place pivot in between the left and right subarays
        arr[i], arr[r] = arr[r], arr[i]
    return i

In [3]:
def qs_tuple(arr,idx,l,r):
    """
    Quick sort a list of tuples in decending order
    
    Args:
        arr: the list of tuples to be sorted
        idx: the index of tuple element to be sorted by
        l: left index of the sublist to be sorted
        r: right index of the sublist to be sorted
    """
    if l >= r: return
    # divide array based on a pivot
    p = divide(arr,idx,l,r)
    # quick sort the subarray left to the pivot (elements <= pivot)
    qs_tuple(arr,idx,l,p-1)
    # quick sort the subarray right to the pivot (elements > pivot)
    qs_tuple(arr,idx,p+1,r)

In [4]:
class Polynomial:
    """
    A class to represent and perform arithmetic operations on polynomials.
    """

    def __init__(self, terms):
        """
        Initializes the Polynomial object from a dict in descending order
        Args:
            terms (dict): Terms of a polynomial represented by key/value,
            where the key is exponent and value coefficient.
        """
        terms = list(terms.items())
        qs_tuple(terms, 0, 0, len(terms)-1)
        self.terms = {k:v for k, v in terms if v != 0 or k == 0}
        if not self.terms:
            self.terms = {0:0}

In [5]:
%%add_to Polynomial
@staticmethod
def _simplify_poly(poly):
    """
    Simplify a polynomial based on representation convention
    """
    def replace(match):
        if match.group(1): # Handles trailing 0x^0
            return ''
        if match.group(2): # Handles signs (+ or -)
            return f' {match.group(2)} '
        if match.group(3): # Handles "1x^0", replacing it with '1'
            return '1'
        if match.group(4) or match.group(5) or match.group(6): # Handles "1x^1|1x|x^1", replacing it with 'x'
            return 'x'
        if match.group(7): # Handles "nx^0", replacing it with ''
            return ''
    pattern = r'( *[+-] *0x\^0$)|([+-])|((?<![\d.])1x\^0(?!\d))|((?<![\d.])1x\^1(?!\d))|((?<![\d.])1x(?!\d))|(x\^1(?!\d))|((?<=\d)x\^0(?!\d))'
    return re.sub(pattern, replace, poly).lstrip()

In [6]:
%%add_to Polynomial
def __str__(self):
    """
    Return a string representation of the polynomial in standard form.
    """
    if not self.terms:
        return '0'

    items = list(self.terms.items())
    poly = f'{items[0][1]:g}x^{items[0][0]}'
    for exp, coeff in items[1:]:
        poly += f'{coeff:+g}x^{exp}'
    return Polynomial._simplify_poly(poly)

In [7]:
%%add_to Polynomial
def __add__(self, other):
    """
    Add two polynomials and return the sum as a new Polynomial.
    """
    poly_sum = self.terms.copy()
    for exp, coeff in other.terms.items():
        poly_sum[exp] = poly_sum.get(exp, 0) + coeff
    return Polynomial(poly_sum)

In [8]:
%%add_to Polynomial
def __sub__(self, other):
    """
    Subtract 'other' polynomial from 'self' and return the difference as a new Polynomial.
    """
    poly_diff = self.terms.copy()
    for exp, coeff in other.terms.items():
        poly_diff[exp] = poly_diff.get(exp, 0) - coeff
    return Polynomial(poly_diff)

In [9]:
%%add_to Polynomial
def __mul__(self, other):
    """
    Multiply two polynomials and return the product as a new Polynomial.
    """
    product = {}
    for exp1, coeff1 in self.terms.items():
        for exp2, coeff2 in other.terms.items():
            exp = exp1 + exp2
            coeff = coeff1 * coeff2
            product[exp] = product.get(exp, 0) + coeff
    return Polynomial(product)

In [10]:
%%add_to Polynomial
def __truediv__(self, other):
    """
    Divides 'self' by 'other' and return a tuple of Polynomials (quotient, remainder).
    """
    q_terms, r_terms = {}, copy.deepcopy(self.terms)
    deg_dvsr_highest = next(iter(other.terms))
    while r_terms:
        # Find the degree and leading coefficient of the quotient
        deg_dvdnd_highest = next(iter(r_terms))
        if deg_dvdnd_highest < deg_dvsr_highest:
            break
        degree_diff = deg_dvdnd_highest - deg_dvsr_highest
        coeff_leading = r_terms[deg_dvdnd_highest] / other.terms[deg_dvsr_highest]
        q_terms[degree_diff] = coeff_leading
        # long division process
        poly = Polynomial({degree_diff:coeff_leading})
        product = poly * other
        degrees = sorted(list(set(r_terms.keys()) | set(product.terms.keys())),
                         reverse=True)
        r_terms = {k: r_terms.get(k, 0) - product.terms.get(k, 0) for k in degrees}
        r_terms = {k: v for k, v in r_terms.items() if v != 0} # remove terms with coeff 0
    return Polynomial(q_terms), Polynomial(r_terms)

In [11]:
%%add_to Polynomial
def evaluate(self, x_value):
    """
    Evaluate the polynomial at a given value of x.

    Args:
        x_value (float): The value at which to evaluate the polynomial.

    Returns:
        float: The result of the polynomial evaluation.
    """
    return sum(
        coeff * (x_value ** exp)
        for exp, coeff in self.terms.items()
    )

In [12]:
def enter_poly(desc = ''):
    """
    Allow users to retrieve a polynomial saved in history or enter a new one
    """
    print(f'Please enter the terms of {desc} (\'q\' to quit):')
    if not history.isEmpty():
        print(f'Use previous polynomial {history.peek()}')
        ans = input('(Y|y = Yes, everything else = No)? ')
        if ans in ('Y', 'y'):
            return history.pop()
    terms = {}
    while True: # enter one term (degree + coefficient) at a time
        try:
            degree = input('  degree: ')
            if degree == 'q':
                if not terms:
                    print('Enter at least one term.')
                    continue
                else:
                    break
            degree = int(degree)
            assert degree >= 0
        except (AssertionError, ValueError):
            print(f'Please enter a positive integer or 0')
            continue
        while True:
            try:
                coeff = float(input('  coefficient: '))
            except ValueError:
                print(f'Please enter a valid number')
                continue
            break
        terms[degree] = terms.get(degree, 0) + coeff
    poly = Polynomial(terms)
    history.push(poly)
    return poly

In [13]:
bold_start = "\033[1m"
red_start = "\033[31m"
blue_start = "\033[34m"
reset_code = "\033[0m"
num_underscores = 50
sym_underscores = '_'

In [14]:
def add_wrapper(p1, p2):
    """
    Wrapper fuction for addtion
    """
    psum = p1 + p2
    history.push(psum)
    print(f'({p1})')
    print(f'+')
    print(f'({p2})')
    print(f'=')
    print(f'{red_start}{psum}{reset_code}')

def sub_wrapper(p1, p2):
    """
    Wrapper fuction for subtraction
    """
    delta = p1 - p2
    history.push(delta)
    print(f'({p1})')
    print(f'-')
    print(f'({p2})')
    print(f'=')
    print(f'{red_start}{delta}{reset_code}')

def mul_wrapper(p1, p2):
    """
    Wrapper fuction for multiplication
    """
    prod = p1 * p2
    history.push(prod)
    print(f'({p1})')
    print(f'*')
    print(f'({p2})')
    print(f'=')
    print(f'{red_start}{prod}{reset_code}')

def div_wrapper(p1, p2):
    """
    Wrapper fuction for division
    """
    if str(p2) == '0':
        print('Error: Divided by 0')
        return
    quotient, remainder = p1 / p2
    history.push(quotient)
    history.push(remainder)
    print(f'({p1})')
    print(f'/')
    print(f'({p2})')
    print(f'=')
    print(f'{blue_start}quotient{reset_code} {red_start}{quotient}{reset_code}')
    print(f'{blue_start}remainder{reset_code} {red_start}{remainder}{reset_code}')

In [15]:
def invalid_op(*_):
    print('Please enter a valid choice')

In [16]:
def perform_arithmetic_ops():
    arithmetic_menu = {'1': ('Addition', add_wrapper),
            '2': ('Subtraction', sub_wrapper),
            '3': ('Multiplication', mul_wrapper),
            '4': ('Division', div_wrapper),
            'q': ('Quit', None)
           }
    ordinals = {0: '1st', 1: '2nd'}
    polys = [None] * len(ordinals)
    for i in range(len(ordinals)):
        polys[i] = enter_poly('the ' + ordinals[i] + ' polynomial')
        print(f'{ordinals[i]} polinomial: {polys[i]}')
    print('\n', end='')
    while True:
        for k, v in arithmetic_menu.items():
            print(f'{k}: {v[0]}')
        op = input('Please select an operation: ')
        print('\n', end='')        
        if op == 'q':
            break
        arithmetic_menu.get(op, (None, invalid_op))[1](polys[0], polys[1])
        print(f'{sym_underscores}'*num_underscores)
        print('\n', end='')

In [17]:
def perform_evaluation():
    Max_Num_Inputs = 5
    inputs = Queue(Max_Num_Inputs)
    done = False
    while not done:
        poly = enter_poly('f(x)')
        print(f'f(x) = {poly}')
        print('\n', end='')
        print(f'Please enter up to {Max_Num_Inputs} x values to be evaluated at (\'q\' to quit):')
        while True:
            try:
                x = input('x =')
                if x == 'q':
                    break
                inputs.enque(float(x))
                if inputs.isFull():
                    done = True
                    break
            except ValueError:
                print(f'Please enter a valid number or \'q\' to quit')
                continue
        print('\n', end='')
        print(f'{bold_start}f(x) = {poly}{reset_code}')
        print(f'{sym_underscores}'*num_underscores)
        print('\n', end='')
        while True:
            x = inputs.deque()
            if x == None:
                break
            poly_eval = poly.evaluate(x)
            print(f'{red_start}f({x:g}) = {poly_eval:g}{reset_code}')
            print(f'{sym_underscores}'*num_underscores)
            print('\n', end='')
        ans = input('Try again (Y|y = Yes, everything else = No)? ')
        if ans not in ('Y', 'y'):
            break

In [18]:
def menu():
    print(f'{bold_start}Polynomial Exercises{reset_code}')
    main_menu = {'1': ('Arithmetic Operations', perform_arithmetic_ops),
                 '2': ('Evaluation', perform_evaluation),
                 'q': ('Quit', None)
                }
    while True:
        for k, v in main_menu.items():
            print(f'{k}: {v[0]}')
        op = input(f'Please select an option: ')
        print('\n', end='')
        if op == 'q':
            break
        main_menu.get(op, (None, invalid_op))[1]()
        print('\n', end='')

In [ ]:
max_history = 10
history = Stack(max_history)
menu()

Polynomial Exercises
1: Arithmetic Operations
2: Evaluation
q: Quit


Please select an option:  1



Please enter the terms of the 1st polynomial ('q' to quit):


  degree:  0
  coefficient:  0
  degree:  q


1st polinomial: 0
Please enter the terms of the 2nd polynomial ('q' to quit):
Use previous polynomial 0


...

Please enter a positive integer or 0


  degree:  3
  coefficient:  -.4
  degree:  0
  coefficient:  44
  degree:  q


2nd polinomial: - 0.4x^3 + 8x + 44

1: Addition
2: Subtraction
3: Multiplication
4: Division
q: Quit


Please select an operation:  1



(0)
+
(- 0.4x^3 + 8x + 44)
=
- 0.4x^3 + 8x + 44
__________________________________________________

1: Addition
2: Subtraction
3: Multiplication
4: Division
q: Quit
